<a href="https://colab.research.google.com/github/Bcaballero95/Ml_Censo/blob/main/Microproyecto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>

# **Microproyecto 2 - Clustering** 
# Censo poblacional en USA entre 2010-2015
---
Introducción a la Inteligencia Artificial

---
**Integrantes:**

* Sergio Páez
* Samuel Huertas
* Brayan Caballero

---
</center>


## Carga del data set

In [7]:
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sb
from tabulate import tabulate 
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import pairwise_distances_argmin_min
import re

%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (25, 12)
plt.style.use('ggplot')

from google.colab import files
import io

uploaded = files.upload()
censo_df = pd.read_csv(io.BytesIO(uploaded["census.csv"]))

Saving census.csv to census (1).csv


## Visualización del dataset

In [ ]:
censo_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,22.949.492,-193.195.585,3.810.660.353,5.820.019.213,-467.369.163,10.300.149.099,8.266.441.875,13.832.821.479,17.247.180.515,7.125.937.237
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,72.420.914.723,-291.592.712,-3.012.348.815,22.659.705.605,-2.530.798.919,76.060.156.669,-2.626.145.543,-2.722.001.942,25.922.703.212,-2.187.333.351
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832.960.211,17.647.292.835,21.845.705.303,19.243.286.504,17.197.872.203,15.844.175.508,1.855.962.658,22.727.626.439,20.317.141.924,18.293.499.115
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728.132.388,-2.500.689.528,-705.682.406,-3.904.216.554,-1.054.329.881,-4.874.741.144,-275.811.345,-7.167.664.228,-3.978.582.583,-1.054.329.881
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527.043.032,-5.068.870.523,-6.201.001.019,-177.537.116,1.772.578.215,-5.088.388.823,-4.363.636.364,-5.403.729.459,7.545.327.445,11.078.613.844


In [ ]:
censo_df.tail()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
3188,50,4,8,56,37,Wyoming,Sweetwater County,43806,43806,43593,...,10.726.430.381,16.243.199.282,-5.339.773.558,-1.425.288.887,-1.424.886.378,12.552.205.765,16.243.199.282,-5.295.460.085,-1.407.528.278,-1.407.019.464
3189,50,4,8,56,39,Wyoming,Teton County,21294,21294,21297,...,-1.589.564.973,9.726.950.601,19.525.928.617,14.143.021.303,-564.849.012,6.545.267.538,24.085.782.441,21.160.657.524,1.630.867.144,15.207.473.387
3190,50,4,8,56,41,Wyoming,Uinta County,21118,21118,21102,...,-177.559.861,-4.916.350.445,-6.902.953.988,-1.421.586.166,-1.212.702.217,-1.813.681.154,-5.536.860.695,-7.521.839.518,-1.474.060.823,-1.260.635.111
3191,50,4,8,56,43,Wyoming,Washakie County,8533,8533,8545,...,-1.163.747.502,-82.781.457,-201.350.231,-1.778.149.054,16.822.879.116,-1.199.012.578,-1.182.592.242,-2.250.384.934,-1.802.016.827,14.419.610.671
3192,50,4,8,56,45,Wyoming,Weston County,7208,7208,7181,...,-1.175.236.097,-8.040.059.243,1.237.258.348,1.533.635.413,69.352.937.097,-1.203.217.908,-8.040.059.243,1.237.258.348,1.533.635.413,69.352.937.097


In [ ]:
censo_df.shape

(3193, 100)

Como se puede observar el dataset cuenta con 100 columnas y con 3193 registros. 

In [ ]:
censo_df.columns

Index(['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME',
       'CENSUS2010POP', 'ESTIMATESBASE2010', 'POPESTIMATE2010',
       'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013',
       'POPESTIMATE2014', 'POPESTIMATE2015', 'NPOPCHG_2010', 'NPOPCHG_2011',
       'NPOPCHG_2012', 'NPOPCHG_2013', 'NPOPCHG_2014', 'NPOPCHG_2015',
       'BIRTHS2010', 'BIRTHS2011', 'BIRTHS2012', 'BIRTHS2013', 'BIRTHS2014',
       'BIRTHS2015', 'DEATHS2010', 'DEATHS2011', 'DEATHS2012', 'DEATHS2013',
       'DEATHS2014', 'DEATHS2015', 'NATURALINC2010', 'NATURALINC2011',
       'NATURALINC2012', 'NATURALINC2013', 'NATURALINC2014', 'NATURALINC2015',
       'INTERNATIONALMIG2010', 'INTERNATIONALMIG2011', 'INTERNATIONALMIG2012',
       'INTERNATIONALMIG2013', 'INTERNATIONALMIG2014', 'INTERNATIONALMIG2015',
       'DOMESTICMIG2010', 'DOMESTICMIG2011', 'DOMESTICMIG2012',
       'DOMESTICMIG2013', 'DOMESTICMIG2014', 'DOMESTICMIG2015', 'NETMIG2010',
       'NETMIG2011', 'NETMIG2012', 'NETMI

Se van a separar los datos utilizados para realizar el clustering, para realizar esta separación se va a observar primero la relación que tienen las variables entre si. 

Cada una de las columnas se explicara a continuación 


*   SUMLEV: es un codigo que utiliza la oficina de censo que indica el tipo geografico. 
*   REGION: territorio donde se realizaron los censos
*   DIVISION: las divisiones censales son agrupaciones de estados que son subdivisiones de las cuatro regiones censales. Hay nueve divisiones censales, que la Oficina del Censo de EE.UU. adoptó en 1910 para la presentación de los datos. Cada división censal se identifica con un código censal de dos dígitos.
*   STATE: es el número del estado.
*   COUNTY: es la divición primaria que se realiza en algunos estados. 
*   STNAME: nombre del estado.
*   CTYNAME: nombre de la ciudad. 
*   CENSUS2010POP: es el censo poblacional realizado en el 2010
*   ESTIMATEBASE2010:
*   POPESTIMATE:
*   NPOPCHG:
*   BIRTHS:
*   DEATHS:
*   NATURALINC:
*   INTERNATIONALMIG:
*   EDOMESTICMIG:
*   NETMIG:
*   RESIDUAL:
*   GQESTIMATEBASE:
*   RBIRTH:
*   RDEATH:
*   RNATURALINC:
*   RINTERATIONALMIG:
*   RDOMESTICMIG:
*   RNETMIG:



## Pre-Procesamiento

### *Análisis de las variables: estado, ciudad, población*

In [2]:
# Se eliminan las columnas con las que no se va a trabajar
# Las columnas seleccionadas para trabajar son STNAME	CTYNAME	POPESTIMATE201x

censo_df_Data=censo_df.drop(['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'COUNTY',
       'CENSUS2010POP', 'ESTIMATESBASE2010', 'NPOPCHG_2010', 'NPOPCHG_2011',
       'NPOPCHG_2012', 'NPOPCHG_2013', 'NPOPCHG_2014', 'NPOPCHG_2015',
       'BIRTHS2010', 'BIRTHS2011', 'BIRTHS2012', 'BIRTHS2013', 'BIRTHS2014',
       'BIRTHS2015', 'DEATHS2010', 'DEATHS2011', 'DEATHS2012', 'DEATHS2013',
       'DEATHS2014', 'DEATHS2015', 'NATURALINC2010', 'NATURALINC2011',
       'NATURALINC2012', 'NATURALINC2013', 'NATURALINC2014', 'NATURALINC2015',
       'INTERNATIONALMIG2010', 'INTERNATIONALMIG2011', 'INTERNATIONALMIG2012',
       'INTERNATIONALMIG2013', 'INTERNATIONALMIG2014', 'INTERNATIONALMIG2015',
       'DOMESTICMIG2010', 'DOMESTICMIG2011', 'DOMESTICMIG2012',
       'DOMESTICMIG2013', 'DOMESTICMIG2014', 'DOMESTICMIG2015', 'NETMIG2010',
       'NETMIG2011', 'NETMIG2012', 'NETMIG2013', 'NETMIG2014', 'NETMIG2015',
       'RESIDUAL2010', 'RESIDUAL2011', 'RESIDUAL2012', 'RESIDUAL2013',
       'RESIDUAL2014', 'RESIDUAL2015', 'GQESTIMATESBASE2010',
       'GQESTIMATES2010', 'GQESTIMATES2011', 'GQESTIMATES2012',
       'GQESTIMATES2013', 'GQESTIMATES2014', 'GQESTIMATES2015', 'RBIRTH2011',
       'RBIRTH2012', 'RBIRTH2013', 'RBIRTH2014', 'RBIRTH2015', 'RDEATH2011',
       'RDEATH2012', 'RDEATH2013', 'RDEATH2014', 'RDEATH2015',
       'RNATURALINC2011', 'RNATURALINC2012', 'RNATURALINC2013',
       'RNATURALINC2014', 'RNATURALINC2015', 'RINTERNATIONALMIG2011',
       'RINTERNATIONALMIG2012', 'RINTERNATIONALMIG2013',
       'RINTERNATIONALMIG2014', 'RINTERNATIONALMIG2015', 'RDOMESTICMIG2011',
       'RDOMESTICMIG2012', 'RDOMESTICMIG2013', 'RDOMESTICMIG2014',
       'RDOMESTICMIG2015', 'RNETMIG2011', 'RNETMIG2012', 'RNETMIG2013',
       'RNETMIG2014', 'RNETMIG2015'],axis=1)


In [3]:
# Se asigna un número a cada estado de los 51 estados registrados en el data set
# se asigna un número a cada ciudad de los 3193 ciudades registrados en el data set

label = preprocessing.LabelEncoder()
censo_df_Data['STNAME'] = label.fit_transform(censo_df_Data['STNAME'].astype(str))
censo_df_Data['CTYNAME'] = label.fit_transform(censo_df_Data['CTYNAME'].astype(str))

In [4]:
# Visualización de los datos
censo_df_Data

,STNAME,CTYNAME,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
0,0,9,4785161,4801108,4816089,4830533,4846411,4858979
1,0,86,54660,55253,55175,55038,55290,55347
2,0,93,183193,186659,190396,195126,199713,203709
3,0,104,27341,27226,27159,26973,26815,26489
4,0,153,22861,22733,22642,22512,22549,22583
...,...,...,...,...,...,...,...,...
3188,50,1676,43593,44041,45104,45162,44925,44626
3189,50,1702,21297,21482,21697,22347,22905,23125
3190,50,1756,21102,20912,20989,21022,20903,20822
3191,50,1819,8545,8469,8443,8443,8316,8328


In [5]:
# Se hace un promedio de la población por cada fila de todos los datos en los 6 años desde el 2010 hasta el 2015

lstprom =[]
for i in censo_df_Data.index:
  prom = 0
  for j in range (2,8):
    s = censo_df_Data.iloc[i][j]
    #out = re.sub(r'[^\w\s]','',s)
    prom += int(s)
  prom = prom/6
  lstprom.append(prom)
#arrprom = np.array(lstprom)
promPOPESTIMATE = pd.DataFrame( lstprom, columns = ['POPESTIMATE'])

censo_df_Data=pd.concat([censo_df_Data.iloc[:,0:2], promPOPESTIMATE], axis = 1)
censo_df_Data

,STNAME,CTYNAME,POPESTIMATE
0,0,9,4.823047e+06
1,0,86,5.512717e+04
2,0,93,1.931327e+05
3,0,104,2.700050e+04
4,0,153,2.264667e+04
...,...,...,...
3188,50,1676,4.457517e+04
3189,50,1702,2.214217e+04
3190,50,1756,2.095833e+04
3191,50,1819,8.424000e+03


In [8]:
fig = px.scatter_3d(censo_df_Data.dropna(), x="STNAME", y="CTYNAME", z="POPESTIMATE")
fig.show()

### *Análisis de las variables: estado, ciudad, nacimientos*

In [9]:
# Se eliminan las columnas con las que no se va a trabajar
# Las columnas seleccionadas para trabajar son STNAME	CTYNAME	BIRTHS201x

censo_df_Data=censo_df.drop(['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'COUNTY',
       'CENSUS2010POP', 'ESTIMATESBASE2010', 'POPESTIMATE2010',
       'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013',
       'POPESTIMATE2014', 'POPESTIMATE2015', 'NPOPCHG_2010', 'NPOPCHG_2011',
       'NPOPCHG_2012', 'NPOPCHG_2013', 'NPOPCHG_2014', 'NPOPCHG_2015',
        'DEATHS2010', 'DEATHS2011', 'DEATHS2012', 'DEATHS2013',
       'DEATHS2014', 'DEATHS2015', 'NATURALINC2010', 'NATURALINC2011',
       'NATURALINC2012', 'NATURALINC2013', 'NATURALINC2014', 'NATURALINC2015',
       'INTERNATIONALMIG2010', 'INTERNATIONALMIG2011', 'INTERNATIONALMIG2012',
       'INTERNATIONALMIG2013', 'INTERNATIONALMIG2014', 'INTERNATIONALMIG2015',
       'DOMESTICMIG2010', 'DOMESTICMIG2011', 'DOMESTICMIG2012',
       'DOMESTICMIG2013', 'DOMESTICMIG2014', 'DOMESTICMIG2015', 'NETMIG2010',
       'NETMIG2011', 'NETMIG2012', 'NETMIG2013', 'NETMIG2014', 'NETMIG2015',
       'RESIDUAL2010', 'RESIDUAL2011', 'RESIDUAL2012', 'RESIDUAL2013',
       'RESIDUAL2014', 'RESIDUAL2015', 'GQESTIMATESBASE2010',
       'GQESTIMATES2010', 'GQESTIMATES2011', 'GQESTIMATES2012',
       'GQESTIMATES2013', 'GQESTIMATES2014', 'GQESTIMATES2015', 'RBIRTH2011',
       'RBIRTH2012', 'RBIRTH2013', 'RBIRTH2014', 'RBIRTH2015', 'RDEATH2011',
       'RDEATH2012', 'RDEATH2013', 'RDEATH2014', 'RDEATH2015',
       'RNATURALINC2011', 'RNATURALINC2012', 'RNATURALINC2013',
       'RNATURALINC2014', 'RNATURALINC2015', 'RINTERNATIONALMIG2011',
       'RINTERNATIONALMIG2012', 'RINTERNATIONALMIG2013',
       'RINTERNATIONALMIG2014', 'RINTERNATIONALMIG2015', 'RDOMESTICMIG2011',
       'RDOMESTICMIG2012', 'RDOMESTICMIG2013', 'RDOMESTICMIG2014',
       'RDOMESTICMIG2015', 'RNETMIG2011', 'RNETMIG2012', 'RNETMIG2013',
       'RNETMIG2014', 'RNETMIG2015'],axis=1)


In [10]:
# Se asigna un número a cada estado de los 51 estados registrados en el data set
# se asigna un número a cada ciudad de los 3193 ciudades registrados en el data set

label = preprocessing.LabelEncoder()
censo_df_Data['STNAME'] = label.fit_transform(censo_df_Data['STNAME'].astype(str))
censo_df_Data['CTYNAME'] = label.fit_transform(censo_df_Data['CTYNAME'].astype(str))

In [11]:
# Visualización de los datos
censo_df_Data

,STNAME,CTYNAME,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015
0,0,9,14226,59689,59062,57938,58334,58305
1,0,86,151,636,615,574,623,600
2,0,93,517,2187,2092,2160,2186,2240
3,0,104,70,335,300,283,260,269
4,0,153,44,266,245,259,247,253
...,...,...,...,...,...,...,...,...
3188,50,1676,167,640,595,657,629,620
3189,50,1702,76,259,230,261,249,269
3190,50,1756,73,324,311,316,316,316
3191,50,1819,26,108,90,95,96,90


In [12]:
# Se hace un promedio de los nacimientos por cada fila de todos los datos en los 6 años desde el 2010 hasta el 2015

lstprom =[]
for i in censo_df_Data.index:
  prom = 0
  for j in range (2,8):
    s = censo_df_Data.iloc[i][j]
    #out = re.sub(r'[^\w\s]','',s)
    prom += int(s)
  prom = prom/6
  lstprom.append(prom)
#arrprom = np.array(lstprom)
promBIRTHS = pd.DataFrame( lstprom, columns = ['BIRTHS'])

censo_df_Data=pd.concat([censo_df_Data.iloc[:,0:2], promBIRTHS], axis = 1)
censo_df_Data

,STNAME,CTYNAME,BIRTHS
0,0,9,51259.000000
1,0,86,533.166667
2,0,93,1897.000000
3,0,104,252.833333
4,0,153,219.000000
...,...,...,...
3188,50,1676,551.333333
3189,50,1702,224.000000
3190,50,1756,276.000000
3191,50,1819,84.166667


In [13]:
fig = px.scatter_3d(censo_df_Data.dropna(), x="STNAME", y="CTYNAME", z="BIRTHS")
fig.show()

### *Análisis de variables: estado, ciudad, muertes*

In [14]:
# Se eliminan las columnas con las que no se va a trabajar
# Las columnas seleccionadas para trabajar son STNAME	CTYNAME	DEATHS201x

censo_df_Data=censo_df.drop(['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'COUNTY',
       'CENSUS2010POP', 'ESTIMATESBASE2010', 'POPESTIMATE2010',
       'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013',
       'POPESTIMATE2014', 'POPESTIMATE2015', 'NPOPCHG_2010', 'NPOPCHG_2011',
       'NPOPCHG_2012', 'NPOPCHG_2013', 'NPOPCHG_2014', 'NPOPCHG_2015',
       'BIRTHS2010', 'BIRTHS2011', 'BIRTHS2012', 'BIRTHS2013', 'BIRTHS2014',
       'BIRTHS2015', 'NATURALINC2010', 'NATURALINC2011',
       'NATURALINC2012', 'NATURALINC2013', 'NATURALINC2014', 'NATURALINC2015',
       'INTERNATIONALMIG2010', 'INTERNATIONALMIG2011', 'INTERNATIONALMIG2012',
       'INTERNATIONALMIG2013', 'INTERNATIONALMIG2014', 'INTERNATIONALMIG2015',
       'DOMESTICMIG2010', 'DOMESTICMIG2011', 'DOMESTICMIG2012',
       'DOMESTICMIG2013', 'DOMESTICMIG2014', 'DOMESTICMIG2015', 'NETMIG2010',
       'NETMIG2011', 'NETMIG2012', 'NETMIG2013', 'NETMIG2014', 'NETMIG2015',
       'RESIDUAL2010', 'RESIDUAL2011', 'RESIDUAL2012', 'RESIDUAL2013',
       'RESIDUAL2014', 'RESIDUAL2015', 'GQESTIMATESBASE2010',
       'GQESTIMATES2010', 'GQESTIMATES2011', 'GQESTIMATES2012',
       'GQESTIMATES2013', 'GQESTIMATES2014', 'GQESTIMATES2015', 'RBIRTH2011',
       'RBIRTH2012', 'RBIRTH2013', 'RBIRTH2014', 'RBIRTH2015', 'RDEATH2011',
       'RDEATH2012', 'RDEATH2013', 'RDEATH2014', 'RDEATH2015',
       'RNATURALINC2011', 'RNATURALINC2012', 'RNATURALINC2013',
       'RNATURALINC2014', 'RNATURALINC2015', 'RINTERNATIONALMIG2011',
       'RINTERNATIONALMIG2012', 'RINTERNATIONALMIG2013',
       'RINTERNATIONALMIG2014', 'RINTERNATIONALMIG2015', 'RDOMESTICMIG2011',
       'RDOMESTICMIG2012', 'RDOMESTICMIG2013', 'RDOMESTICMIG2014',
       'RDOMESTICMIG2015', 'RNETMIG2011', 'RNETMIG2012', 'RNETMIG2013',
       'RNETMIG2014', 'RNETMIG2015'],axis=1)

In [15]:
# Se asigna un número a cada estado de los 51 estados registrados en el data set
# se asigna un número a cada ciudad de los 3193 ciudades registrados en el data set

label = preprocessing.LabelEncoder()
censo_df_Data['STNAME'] = label.fit_transform(censo_df_Data['STNAME'].astype(str))
censo_df_Data['CTYNAME'] = label.fit_transform(censo_df_Data['CTYNAME'].astype(str))

In [16]:
# Visualización de los datos
censo_df_Data

,STNAME,CTYNAME,DEATHS2010,DEATHS2011,DEATHS2012,DEATHS2013,DEATHS2014,DEATHS2015
0,0,9,11089,48811,48357,50843,50228,50330
1,0,86,152,507,558,583,504,467
2,0,93,532,1825,1879,1902,2044,1992
3,0,104,128,319,291,294,310,309
4,0,153,34,278,237,281,211,223
...,...,...,...,...,...,...,...,...
3188,50,1676,76,251,273,296,246,262
3189,50,1702,10,87,61,97,68,76
3190,50,1756,49,139,115,136,130,137
3191,50,1819,34,79,105,77,70,79


In [17]:
# Se hace un promedio de las muertes por cada fila de todos los datos en los 6 años desde el 2010 hasta el 2015

lstprom =[]
for i in censo_df_Data.index:
  prom = 0
  for j in range (2,8):
    s = censo_df_Data.iloc[i][j]
    #out = re.sub(r'[^\w\s]','',s)
    prom += int(s)
  prom = prom/6
  lstprom.append(prom)
#arrprom = np.array(lstprom)
promDEATHS = pd.DataFrame( lstprom, columns = ['DEATHS'])

censo_df_Data=pd.concat([censo_df_Data.iloc[:,0:2], promDEATHS], axis = 1)
censo_df_Data

,STNAME,CTYNAME,DEATHS
0,0,9,43276.333333
1,0,86,461.833333
2,0,93,1695.666667
3,0,104,275.166667
4,0,153,210.666667
...,...,...,...
3188,50,1676,234.000000
3189,50,1702,66.500000
3190,50,1756,117.666667
3191,50,1819,74.000000


In [18]:
fig = px.scatter_3d(censo_df_Data.dropna(), x="STNAME", y="CTYNAME", z="DEATHS")
fig.show()

## Metodos para establecer el número de clusteres

## Agrupamiento Jeraárquico

## Análisis de Resultados 

In [ ]:
print(censo_df.groupby('STNAME').size())

In [ ]:
arrayState = np.array(censo_df['STNAME'].unique())
print(len(arrayState))
arrayCity = np.array(censo_df['CTYNAME'].unique())
print(len(arrayCity))

In [ ]:
# Se va a realizar el gráfico de dispersión de la estimación de la población durante los cinco años 
sb.pairplot(censo_df.dropna(),
            height=4, 
            vars=[
       'CENSUS2010POP', 'ESTIMATESBASE2010', 'POPESTIMATE2010',
       'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013',
       'POPESTIMATE2014', 'POPESTIMATE2015'],
            kind='scatter')
plt.show()

In [ ]:
# Se va a realizar el gráfico de dispersión de la estimación de la población durante los cinco años 
sb.pairplot(censo_df.dropna(),
            height=4, 
            vars=[
       'POPESTIMATE2010',
       'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013',
       'POPESTIMATE2014', 'POPESTIMATE2015'],
            kind='scatter')
plt.show()